<a href="https://colab.research.google.com/github/KRiver28/prac/blob/master/Cafe_Transformer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [11]:
%cd /content/drive/MyDrive/BIGDATA_STUDY/project_NLP/transform_test

/content/drive/MyDrive/BIGDATA_STUDY/project_NLP/transform_test


In [12]:
from Transformer import *


import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import urllib.request
import time
import tensorflow_datasets as tfds
import tensorflow as tf

In [13]:
##############################################
########파일 불러오기(Q, A가 있는 csv형태여야함.)
##################################

#csv 파일 불러오기



# urllib.request.urlretrieve("https://raw.githubusercontent.com/songys/Chatbot_data/master/ChatbotData.csv", filename="ChatBotData.csv")

train_data = pd.read_csv('/content/drive/MyDrive/BIGDATA_STUDY/project_NLP/KoGPT2-chatbot-master/koGPT2_test/Chatbot_data/ChatbotData.csv')
train_data

,Unnamed: 0,Q,A,label
0,0,빙수 위에 뿌려진 과일에서 쉰 냄새가 나는데 어떻게 하면 되죠?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
1,1,마카롱 주문했는데 마카롱이 엄청 딱딱해요.,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
2,2,와플 지금 받았는데 너무 눅눅해서 못먹겠어요 이거 어떻게 하면 되나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
3,3,케이크에 생크림 모양이 너무 변형된 것 같은데 어떻게 하나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
4,5,과일이 좀 씁쓸한데 이거 다 익은 거 맞나요?,죄송합니다. 카운터에서 확인 도와드리겠습니다.,41
...,...,...,...,...
35933,3727,얼마인가요?,어떤 메뉴 말씀이신가요?,72
35934,3728,얼마죠?,어떤 메뉴 말씀이신가요?,72
35935,3729,메뉴랑 가격 알려주세요,"아메리카노: 3500원, 카페라떼: 4500원, 카페모카: 5500원, 바닐라라떼:...",72
35936,3730,메뉴 가격,"아메리카노: 3500원, 카페라떼: 4500원, 카페모카: 5500원, 바닐라라떼:...",72


In [14]:
######################
######전처리 작업
######################

questions = []
for sentence in train_data['Q']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    questions.append(sentence)


answers = []
for sentence in train_data['A']:
    # 구두점에 대해서 띄어쓰기
    # ex) 12시 땡! -> 12시 땡 !
    sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
    sentence = sentence.strip()
    answers.append(sentence)


tokenizer = tfds.deprecated.text.SubwordTextEncoder.build_from_corpus(
    questions + answers, target_vocab_size=2**13)




# 시작 토큰과 종료 토큰에 대한 정수 부여.
START_TOKEN, END_TOKEN = [tokenizer.vocab_size], [tokenizer.vocab_size + 1]

# 시작 토큰과 종료 토큰을 고려하여 단어 집합의 크기를 + 2
VOCAB_SIZE = tokenizer.vocab_size + 2



# 최대 길이를 40으로 정의
MAX_LENGTH = 40

# 토큰화 / 정수 인코딩 / 시작 토큰과 종료 토큰 추가 / 패딩
def tokenize_and_filter(inputs, outputs):
  tokenized_inputs, tokenized_outputs = [], []

  for (sentence1, sentence2) in zip(inputs, outputs):
    # encode(토큰화 + 정수 인코딩), 시작 토큰과 종료 토큰 추가
    sentence1 = START_TOKEN + tokenizer.encode(sentence1) + END_TOKEN
    sentence2 = START_TOKEN + tokenizer.encode(sentence2) + END_TOKEN

    tokenized_inputs.append(sentence1)
    tokenized_outputs.append(sentence2)

  # 패딩
  tokenized_inputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_inputs, maxlen=MAX_LENGTH, padding='post')
  tokenized_outputs = tf.keras.preprocessing.sequence.pad_sequences(
      tokenized_outputs, maxlen=MAX_LENGTH, padding='post')

  return tokenized_inputs, tokenized_outputs
questions, answers = tokenize_and_filter(questions, answers)



# 텐서플로우 dataset을 이용하여 셔플(shuffle)을 수행하되, 배치 크기로 데이터를 묶는다.
# 또한 이 과정에서 교사 강요(teacher forcing)을 사용하기 위해서 디코더의 입력과 실제값 시퀀스를 구성한다.
BATCH_SIZE = 64
BUFFER_SIZE = 20000

# 디코더의 실제값 시퀀스에서는 시작 토큰을 제거해야 한다.
dataset = tf.data.Dataset.from_tensor_slices((
    {
        'inputs': questions,
        'dec_inputs': answers[:, :-1] # 디코더의 입력. 마지막 패딩 토큰이 제거된다.
    },
    {
        'outputs': answers[:, 1:]  # 맨 처음 토큰이 제거된다. 다시 말해 시작 토큰이 제거된다.
    },
))

dataset = dataset.cache()
dataset = dataset.shuffle(BUFFER_SIZE)
dataset = dataset.batch(BATCH_SIZE)
dataset = dataset.prefetch(tf.data.experimental.AUTOTUNE)

In [15]:
################################
#########transformer 모델 만들기
################################

# 하이퍼파라미터
D_MODEL = 256
NUM_LAYERS = 2
NUM_HEADS = 8
DFF = 512
DROPOUT = 0.1
model = transformer(
    vocab_size=VOCAB_SIZE,
    num_layers=NUM_LAYERS,
    dff=DFF,
    d_model=D_MODEL,
    num_heads=NUM_HEADS,
    dropout=DROPOUT)



learning_rate = CustomSchedule(D_MODEL)

optimizer = tf.keras.optimizers.Adam(
    learning_rate, beta_1=0.9, beta_2=0.98, epsilon=1e-9)

def loss_function(y_true, y_pred):
    
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))

  loss = tf.keras.losses.SparseCategoricalCrossentropy(
      from_logits=True, reduction='none')(y_true, y_pred)

  mask = tf.cast(tf.not_equal(y_true, 0), tf.float32)
  loss = tf.multiply(loss, mask)

  return tf.reduce_mean(loss)


def accuracy(y_true, y_pred):
  # 레이블의 크기는 (batch_size, MAX_LENGTH - 1)
  y_true = tf.reshape(y_true, shape=(-1, MAX_LENGTH - 1))
  return tf.keras.metrics.sparse_categorical_accuracy(y_true, y_pred)

model.compile(optimizer=optimizer, loss=loss_function, metrics=[accuracy])

model.summary()

(1, 7901, 256)
(1, 7901, 256)
Model: "transformer"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 inputs (InputLayer)            [(None, None)]       0           []                               
                                                                                                  
 dec_inputs (InputLayer)        [(None, None)]       0           []                               
                                                                                                  
 enc_padding_mask (Lambda)      (None, 1, 1, None)   0           ['inputs[0][0]']                 
                                                                                                  
 encoder (Functional)           (None, None, 256)    3076864     ['inputs[0][0]',                 
                                                          

In [16]:
#학습시키기
EPOCHS = 50
model.fit(dataset, epochs=EPOCHS)

Epoch 1/50
562/562 [==============================] - 76s 121ms/step - loss: 1.2930 - accuracy: 0.0663
Epoch 2/50
562/562 [==============================] - 68s 121ms/step - loss: 0.7202 - accuracy: 0.1132
Epoch 3/50
562/562 [==============================] - 68s 120ms/step - loss: 0.5976 - accuracy: 0.1237
Epoch 4/50
562/562 [==============================] - 67s 120ms/step - loss: 0.5050 - accuracy: 0.1314
Epoch 5/50
562/562 [==============================] - 68s 120ms/step - loss: 0.4316 - accuracy: 0.1379
Epoch 6/50
562/562 [==============================] - 68s 121ms/step - loss: 0.3715 - accuracy: 0.1435
Epoch 7/50
562/562 [==============================] - 68s 120ms/step - loss: 0.3233 - accuracy: 0.1486
Epoch 8/50
562/562 [==============================] - 68s 120ms/step - loss: 0.2814 - accuracy: 0.1537
Epoch 9/50
562/562 [==============================] - 68s 120ms/step - loss: 0.2314 - accuracy: 0.1610
Epoch 10/50
562/562 [==============================] - 68s 120ms/step - l

In [17]:
########인풋, 아웃풋으로 구현해보기
def preprocess_sentence(sentence):
  # 단어와 구두점 사이에 공백 추가.
  # ex) 12시 땡! -> 12시 땡 !
  sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
  sentence = sentence.strip()
  return sentence

def evaluate(sentence):
  # 입력 문장에 대한 전처리
  sentence = preprocess_sentence(sentence)

  # 입력 문장에 시작 토큰과 종료 토큰을 추가
  sentence = tf.expand_dims(
      START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

  output = tf.expand_dims(START_TOKEN, 0)

  # 디코더의 예측 시작
  try:
      for i in range(MAX_LENGTH):
        predictions = model(inputs=[sentence, output], training=False)

        # 현재 시점의 예측 단어를 받아온다.
        predictions = predictions[:, -1:, :]
        predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

        # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
        if tf.equal(predicted_id, END_TOKEN[0]):
          break

       # 현재 시점의 예측 단어를 output(출력)에 연결한다.
       # output은 for문의 다음 루프에서 디코더의 입력이 된다.
        output = tf.concat([output, predicted_id], axis=-1)

      # 단어 예측이 모두 끝났다면 output을 리턴.
      return tf.squeeze(output, axis=0)
  except:
    print("죄송합니다.")

def predict(sentence):
  prediction = evaluate(sentence)

  # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
  # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
  predicted_sentence = tokenizer.decode(
      [i for i in prediction if i < tokenizer.vocab_size])


  # print('Input: {}'.format(sentence))
  # print('Output: {}'.format(predicted_sentence))

  return predicted_sentence

def chatting(n=100):
    for i in range(n):
        question = input('Q : ')
        
        if  question == 'quit':
            break
        
        print('A :', predict(question) )

In [18]:
# ########인풋, 아웃풋으로 구현해보기
# def preprocess_sentence(sentence):
#   # 단어와 구두점 사이에 공백 추가.
#   # ex) 12시 땡! -> 12시 땡 !
#   sentence = re.sub(r"([?.!,])", r" \1 ", sentence)
#   sentence = sentence.strip()
#   return sentence

# def evaluate(sentence):
#   # 입력 문장에 대한 전처리
#   sentence = preprocess_sentence(sentence)

#   # 입력 문장에 시작 토큰과 종료 토큰을 추가
#   sentence = tf.expand_dims(
#       START_TOKEN + tokenizer.encode(sentence) + END_TOKEN, axis=0)

#   output = tf.expand_dims(START_TOKEN, 0)

#   # 디코더의 예측 시작
  
#   for i in range(MAX_LENGTH):
#     predictions = model(inputs=[sentence, output], training=False)

#     # 현재 시점의 예측 단어를 받아온다.
#     predictions = predictions[:, -1:, :]
#     predicted_id = tf.cast(tf.argmax(predictions, axis=-1), tf.int32)

#     # 만약 현재 시점의 예측 단어가 종료 토큰이라면 예측을 중단
#     if tf.equal(predicted_id, END_TOKEN[0]):
#       break

#     # 현재 시점의 예측 단어를 output(출력)에 연결한다.
#     # output은 for문의 다음 루프에서 디코더의 입력이 된다.
#     output = tf.concat([output, predicted_id], axis=-1)
#     if i== MAX_LENGTH:
#       output = "죄송합니다"
#       break
#   # 단어 예측이 모두 끝났다면 output을 리턴.
#   return tf.squeeze(output, axis=0)

# def predict(sentence):
#   prediction = evaluate(sentence)

#   # prediction == 디코더가 리턴한 챗봇의 대답에 해당하는 정수 시퀀스
#   # tokenizer.decode()를 통해 정수 시퀀스를 문자열로 디코딩.
#   predicted_sentence = tokenizer.decode(
#       [i for i in prediction if i < tokenizer.vocab_size])


#   # print('Input: {}'.format(sentence))
#   # print('Output: {}'.format(predicted_sentence))

#   return predicted_sentence

# def chatting(n=100):
#     for i in range(n):
#         question = input('Q : ')
        
#         if  question == 'quit':
#             break
        
#         print('A :', predict(question) )

In [21]:
model.save_weights('transformer_model.h5')

In [22]:
####### Chatting 시작 #######
print("\nSeq2Seq ChatBot (ver. 1.0)")
print("Chatting 모듈을 로드하고 있습니다 ...")
# 채팅을 시작한다.
chatting(100)


Seq2Seq ChatBot (ver. 1.0)
Chatting 모듈을 로드하고 있습니다 ...
Q : 안녕하세요
A : 어서오세요 ,  Learning Crew Café 입니다 .  무엇을 도와드릴까요 ?
Q : 먹을것도 파나요?
A : 빙수 ,  케이크 ,  쿠키 ,  아이스크림 ,  초콜릿 ,  와플 ,  도넛 ,  마카롱도 판매중입니다 .
Q : 메뉴
A : 네
Q : 메뉴알려주세요
A : 시즌 메뉴입니다
Q : 메뉴는 뭐가 있나요?
A : 커피도 있고 디저트도 있고 여러가지 있습니다
Q : 음료 메뉴는 뭐가 있나요?
A : 음료 메뉴는 아메리카노 ,  카페라떼 ,  카페모카 ,  바닐라라떼 ,  콜드브루 ,  카라멜 마끼아또 ,  카푸치노 ,  밀크티가 있습니다 .
Q : 마실 것 뭐가 있나요?
A : 음료 메뉴는 아메리카노 ,  카페라떼 ,  카페모카 ,  바닐라라떼 ,  콜드브루 ,  카라멜 마끼아또 ,  카푸치노 ,  밀크티가 있습니다 .
Q : 메뉴 가격
A : 블루베리 확인 되셨던 카드 주시면 
Q : 메뉴랑 가격 알려주세요
A : 곱게 갈은 냅니다
Q : 메뉴판
A : 천천히 보세요
Q : 저기요
A : 어서오세요 ,  Learning Crew Café 입니다 .  무엇을 도와드릴까요 ?
Q : 아아 한잔 주세요
A : 네 ,  아이스 아메리카노 한 잔 준비해드리겠습니다 .
Q : 카페라떼 3잔이랑 도넛 같이 주문하면 얼마인가요?
A : 총 15700원 입니다 .
Q : 카페라떼 3잔이랑 카푸치노 2잔 주문하면 얼마인가요?
A : 총 23500원 입니다 .
Q : 카페라떼 3잔이랑 카푸치노 5잔 주문하면 얼마인가요?
A : 총 38500원 입니다 .
Q : 안녕하세요
A : 어서오세요 ,  Learning Crew Café 입니다 .  무엇을 도와드릴까요 ?
Q : 빙수 주세요
A : 네
Q : 빙수 얼마에요?
A : 빙수 가격은 9800원 입니다 .
Q : 캐리어에 담아주세요
A : 네 알겠습니다 .
Q : 아메리카노 2잔, 카페라떼 1